[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DSIMB/PoincareMSA/blob/master/PoincareMSA_colab.ipynb)

<img src="https://github.com/DSIMB/PoincareMSA/blob/master/.github/PoincareMSA_small_logo.png?raw=true" height="100" style="height:100px;margin-left: 0px;">

# Poincaré maps for visualization of large protein famillies

**Authors**: Anna Klimovskaia Susmelj, Yani Ren, Yann Vander Meersche, Jean-Christophe Gelly and Tatiana Galochkina

PoincaréMSA builds an interactive projection of an input protein multiple sequence alignemnt (MSA) using a method based on Poincaré maps described by Klimovskaia et al [1]. It reproduces both local proximities of protein sequences and hierarchy contained in give data. Thus, sequences located closer to the center of projection correspond to the proteins sharing the most general functional properites and/or appearing at the earlier stages of evolution. Source code is available at https://github.com/DSIMB/PoincareMSA.

[1] Klimovskaia, A., Lopez-Paz, D., Bottou, L. et al. Poincaré maps for analyzing complex hierarchies in single-cell data. Nat Commun 11, 2966 (2020).

# Notebook initialization

In [1]:
# Update working directory

%cd ../../../

/Users/gabrielduval/Desktop/Paris_Cite/Year2/Projet_Long/PoincareMSA


In [2]:
#Load dependencies
import os
import numpy as np
import pandas as pd
import subprocess
import json
import warnings
warnings.filterwarnings('ignore')

#Import visualization functions
from scripts.visualize_projection.pplots_new import read_embeddings, plot_embedding, plot_embedding_interactive, rotate, get_colors
from scripts.prepare_data.mmseqs2_api import run_mmseqs2
from scripts.prepare_data.uniprot_idmapping_api import submit_id_mapping, check_id_mapping_results_ready, get_id_mapping_results_link, get_id_mapping_results_search
%matplotlib inline

#Create optional variables
path_annotation = ""

# Data import

In [3]:
# OPTIONS =================================================
# Path to MSA in mfasta format OR folder containing the embeddings - leave the other one as None
mfasta = 'examples/wayland_datasets/radical_sam/radical_sam.mfasta'
embedding_path = None #'embeddings/ankh_base_radical_sam/'

#Annotation file (.csv) or UniProt ID list. (Emtpy strings for no annotations)
path_annotation_csv = "examples/wayland_datasets/radical_sam/radicalsam_labels.csv"   # Path or ""
# OR
path_uniprot_list = ""   # Path or ""
#==========================================================


#Check files
#mfasta
nb_seq = 0
if mfasta is not None and embedding_path is not None:
    print('You cannot import both an mfasta and an embedding folder ! Choose one and leave the other as None.')
elif mfasta is not None:
    if os.path.isfile(mfasta):
        with open(mfasta, "r") as f_in:
            for line in f_in:
                if line[0] == ">":
                    nb_seq += 1
        print(f"\nNumber of sequences found: {nb_seq}.")
    else:
        print(f"File {mfasta} not found.")
elif embedding_path is not None:
    if os.path.exists(embedding_path):
        nb_seq = len([s for s in os.listdir(embedding_path) if '.pt' in s])
        print(f"\nNumber of sequences found: {nb_seq}.")
    else:
        print(f"Folder {embedding_path} not found.")
else:
    print('Neither a valid mfasta or embedding folder has been provided')

#Check that only one path is selected
if path_annotation_csv and path_uniprot_list:
    raise ValueError("Use only one file path (path_annotation_csv OR path_uniprot_list).")

#Check that only one path is selected
if path_annotation_csv and path_uniprot_list:
    raise ValueError("Use only one file path (path_annotation_csv OR path_uniprot_list).")

if path_annotation_csv:
    if os.path.isfile(path_annotation_csv):
        try:
            df_annotation = pd.read_csv(path_annotation_csv)
        except:
            raise ValueError("Annotation file is not in .csv format.")
        else:
            if len(df_annotation) != nb_seq:
                raise ValueError("Annotation file doesn't match the .mfasta file length.")

        #Add id column
        if "proteins_id" not in df_annotation.columns:
            df_annotation.insert(0, "proteins_id", range(len(df_annotation)))
        path_annotation = path_annotation_csv
            
        print("\nAnnotation file correctly loaded.")
        annotation_names = list(df_annotation.columns)
        print(f"{len(annotation_names)} annotations found: {annotation_names}.")
    else:
        print(f"File {path_annotation_csv} not found.")

elif path_uniprot_list:
    if os.path.isfile(path_uniprot_list):
        try:
            UnP_ids = np.genfromtxt(path_uniprot_list, dtype="str")
        except:
            raise ValueError("UniProt IDs file is not in a valid format.")
        else:
            if len(UnP_ids) != nb_seq:
                raise ValueError("UniProt IDs file doesn't match the .mfasta file length.")     

            #Split UniProtKB and UniParc IDs
            uniparc_ids = []
            uniprot_ids = []
            for unp in UnP_ids:
                if len(unp) == 13 and unp[:2] == "UP":
                    uniparc_ids.append(unp)
                else:
                    uniprot_ids.append(unp)

            #Fetch UniProtKB annotations
            job_id = submit_id_mapping(
                from_db="UniProtKB_AC-ID", to_db="UniParc", ids=uniprot_ids
            )

            if check_id_mapping_results_ready(job_id):
                link = get_id_mapping_results_link(job_id)
                results = get_id_mapping_results_search(link)

            #Fetch UniParc annotations
            job_id = submit_id_mapping(
                from_db="UniParc", to_db="UniParc", ids=uniparc_ids
            )

            if check_id_mapping_results_ready(job_id):
                link = get_id_mapping_results_link(job_id)
                results2 = get_id_mapping_results_search(link)

            #Create annotation dataframe
            df_annotation = pd.DataFrame(UnP_ids, columns=["UnP_ID"])
            df_annotation[["organism", "proteinName", "taxonId", "species", "genus", \
                           "family", "order", "class", "phylum", "clade", "superkingdom"]] = ""

            #Fill the annotation DataFrame
            for dict_res in results["results"] + results2["results"]:
                try:
                    unp = dict_res["from"]
                    prot_name = dict_res["to"]["uniParcCrossReferences"][0]["proteinName"]
                    df_annotation.loc[df_annotation["UnP_ID"] == unp, "proteinName"] = prot_name
                    scientific_name = dict_res["to"]["uniParcCrossReferences"][0]["organism"]["scientificName"]
                    taxid = dict_res["to"]["uniParcCrossReferences"][0]["organism"]["taxonId"]
                    df_annotation.loc[df_annotation["UnP_ID"] == unp, "organism"] = scientific_name
                    df_annotation.loc[df_annotation["UnP_ID"] == unp, "taxonId"] = taxid
                except KeyError:
                    continue

            #Add lineage from NCBI Taxonomist
            taxon_ids = df_annotation.loc[df_annotation["taxonId"].notnull(), 'taxonId'].to_numpy()
            taxon_ids = list(set(taxon_ids))
            taxon_ids = list(map(str, taxon_ids))
            bash_command = f"ncbi-taxonomist resolve -t {','.join(taxon_ids)}"
            list_taxon = subprocess.run(bash_command, shell=True, capture_output=True, text=True).stdout.strip().split("\n")

            if list_taxon != [""]:
                for taxon in list_taxon:
                    jsonString = taxon
                    taxon_dict = json.loads(jsonString)
                    query = taxon_dict["query"]
                    for lineage in taxon_dict["lineage"]:
                        rank = lineage["rank"]
                        if rank in ["species", "genus", "family", "order", "class", "phylum", "clade", "superkingdom"]:
                            name = lineage["name"]
                            df_annotation.loc[df_annotation["taxonId"] == int(query), rank] = name

            #Add id column
            if "proteins_id" not in df_annotation.columns:
                df_annotation.insert(0, "proteins_id", range(len(df_annotation)))

            #Save annotation to csv
            path_annotation = "auto_annot.csv"
            df_annotation.to_csv(path_annotation, index=False)

            print("\nAnnotation correctly fetched.")
            annotation_names = list(df_annotation.columns)
            print(f"{len(annotation_names)} annotations found: {annotation_names}.")
    else:
        print(f"File {path_uniprot_list} not found.")
else:
    print("No annotation file selected.")
    path_annotation = ""


Number of sequences found: 179.

Annotation file correctly loaded.
3 annotations found: ['proteins_id', 'accession', 'label'].


# Data preparation
Here we clean the input .mfasta alignment and translate each sequence to a vector ready for projection.

### Parameters for data preparation

In [4]:
# OPTIONS =================================================
# Job name
#Name for the output folder
out_name = "radical_sam_data"
#----------------------------------------------------------
# Threshold for filtering gapped positions
#Positions with proportion of gaps above the given threshold are removed from the alignment.
#If your alignment is very gapped, you may want to increase this value.
gapth = 0.9 
#==========================================================


# Run data preparation
#Data preparation consists in `.mfasta` cleaning according to a gap threshold and
#translation of each sequence to the PSSM profile.
prep_parameters = "scripts/prepare_data" + " " + mfasta + " " + out_name + " " + out_name + " " + str(gapth)
bash_projection = "bash scripts/prepare_data/create_projection.sh " + prep_parameters
!{bash_projection}

Input file: examples/wayland_datasets/radical_sam/radical_sam.mfasta
Name of the protein family: radical_sam
filter_gaps finished for examples/wayland_datasets/radical_sam/radical_sam.mfasta
mfasta2fasta finished for radical_sam_data/radical_sam_data.clean0.9.mfasta


# Projection

### Projection parameters

In [8]:
### Projection parameters (no embeddings)
# EMBEDDING SELECTION
in_name = 'radical_sam_data/fasta0.9/'  # Input here the name of folder with the fasta files
mid_output = 'radical_sam_data/with_mfasta/'  # Input name of desired folder for intermediary results
out_name = 'results/radical_sam/with_mfasta/' # Input desired name of output folder

# OPTIONS =================================================
#Here you control different parameters of Poincaré maps.
#In our computational experiments the best results were achieved for the following values provided by default.
#The impact of different parameters is analyzed in the original paper [1].
knn = 5
gamma = 2
sigma = 1
cospca = 0
batchs = 4
epochs = 1000
seed = 4
#==========================================================


# Building projection and preparing data for visualization
#This step creates a projection of encoded sequences to a Poincaré disk.
# bash_pm = "python3 "+ "scripts/build_poincare_map/main.py --input_path " + out_name + "/fasta" + str(gapth) + " --output_path " + out_name + "/projections/ --gamma "+ str(gamma) +" --pca "+ str(cospca) + " --epochs "+ str(epochs) +" --seed "+ str(seed) + " --knn " + str(knn)
# !{bash_pm}
bash_pm = "python "+ "scripts/build_poincare_map/main.py --input_path " + in_name + \
          " --output_path " + out_name + " --plm_embedding False" +  " --matrices_output_path " + mid_output \
          + " --gamma "+ str(gamma) +" --pca "+ str(cospca) + " --epochs "+ str(epochs) +" --seed "\
          + str(seed) + " --knn " + str(knn)
!{bash_pm}

CUDA: False
Random seed set as 4
179 proteins found in folder radical_sam_data/fasta0.9/.
No root detected
142.aamtx
Prepare data: tensor construction
Prepare data: successfully terminated
labels: ['142' '107' '55' '10' '95' '68' '71' '34' '166' '123' '164' '121' '73'
 '36' '159' '138' '97' '57' '12' '140' '105' '93' '179' '8' '53' '16'
 '144' '101' '160' '125' '77' '32' '118' '48' '75' '30' '162' '127' '88'
 '146' '103' '51' '14' '29' '91' '158' '120' '165' '37' '72' '13' '56'
 '104' '141' '139' '96' '94' '69' '106' '143' '11' '54' '35' '70' '122'
 '167' '31' '74' '126' '163' '89' '49' '28' '90' '102' '147' '15' '50'
 '17' '52' '100' '145' '9' '92' '178' '119' '124' '161' '33' '76' '87'
 '128' '47' '150' '115' '174' '131' '63' '26' '149' '5' '19' '7' '61' '24'
 '176' '133' '152' '117' '45' '78' '85' '156' '113' '41' '81' '39' '3'
 '58' '98' '65' '20' '172' '137' '170' '135' '67' '22' '1' '108' '169'
 '83' '43' '154' '111' '25' '60' '132' '177' '6' '18' '79' '84' '116'
 '153' '44' '46'

### Projection parameters (with embeddings)

In [23]:
### Projection parameters (with embeddings)
# EMBEDDING SELECTION
in_name = 'embeddings/ankh_base_radical_sam/'  # Input here the name of folder with the embeddings
mid_output = 'radical_sam_data/with_plm_embeddings/'  # Input name of desired folder for intermediary results
out_name = 'results/radical_sam/with_plm_embeddings/' # Input desired name of output folder


# OPTIONS =================================================
#Here you control different parameters of Poincaré maps.
#In our computational experiments the best results were achieved for the following values provided by default.
#The impact of different parameters is analyzed in the original paper [1].
knn = 5
gamma = 4
sigma = 1
cospca = 0
batchs = 4
epochs = 1000
seed = 4
#==========================================================


# Building projection and preparing data for visualization
#This step creates a projection of encoded sequences to a Poincaré disk.
bash_pm = "python "+ "scripts/build_poincare_map/main.py --input_path " + in_name + \
          " --output_path " + out_name + " --plm_embedding True" +  " --matrices_output_path " + mid_output +\
          " --distlocal minkowski" + " --gamma "+ str(gamma) +" --pca "+ str(cospca) + " --epochs "+ str(epochs) +" --seed "\
          + str(seed) + " --knn " + str(knn)
!{bash_pm}

CUDA: False
Random seed set as 4
179 proteins found in folder embeddings/ankh_base_radical_sam/.
['19.pt', '48.pt', '115.pt', '144.pt', '170.pt', '121.pt', '131.pt', '160.pt', '154.pt', '105.pt', '58.pt', '29.pt', '78.pt', '125.pt', '174.pt', '140.pt', '111.pt', '101.pt', '150.pt', '164.pt', '135.pt', '68.pt', '39.pt', '4.pt', '28.pt', '79.pt', '124.pt', '175.pt', '141.pt', '110.pt', '100.pt', '151.pt', '165.pt', '134.pt', '69.pt', '38.pt', '5.pt', '18.pt', '49.pt', '114.pt', '145.pt', '171.pt', '120.pt', '130.pt', '161.pt', '155.pt', '104.pt', '59.pt', '1.pt', '76.pt', '27.pt', '85.pt', '13.pt', '42.pt', '52.pt', '95.pt', '37.pt', '66.pt', '46.pt', '17.pt', '23.pt', '72.pt', '81.pt', '91.pt', '62.pt', '33.pt', '56.pt', '47.pt', '16.pt', '22.pt', '73.pt', '80.pt', '90.pt', '63.pt', '32.pt', '57.pt', '77.pt', '26.pt', '84.pt', '12.pt', '43.pt', '53.pt', '94.pt', '36.pt', '67.pt', '119.pt', '148.pt', '15.pt', '44.pt', '83.pt', '70.pt', '21.pt', '31.pt', '60.pt', '93.pt', '54.pt', '158.pt

# Projection visualization

### Prepare data for visualization

In [ ]:
# Prepare data for visualization
out_name = 'results/radical_sam/with_plm_embeddings'

#Check that an annotation file was provided. Create a dummy one instead
if not path_annotation:
    df_annotation = pd.DataFrame(list(zip(list(range(1,nb_seq+1)), np.full(nb_seq, "-", dtype=object))), columns=["id", "default"])
    df_annotation.to_csv("dummy_annotation.csv", index=False)
    path_annotation = "dummy_annotation.csv"
    annotation_names = ["proteins_id"]

path_embedding = f"{out_name}/PM{knn:1.0f}sigma={sigma:2.2f}gamma={gamma:2.2f}cosinepca={cospca:1.0f}_seed{seed:1.0f}.csv"
df_embedding = read_embeddings(path_embedding, path_annotation, withroot=False)

#Here are different labels found in your annotation file (if one uploaded):
print(f"{len(annotation_names)} annotations found: {annotation_names}.")

          pm1       pm2                                        proteins_id
0    0.546341  0.133484  A0A0H3KB22 3a6b QueE-like_QueE Bacteria_Bmulti...
1    0.571726  0.611394         A0A0R0FIC1 6a6b TW-synth_TYW1 Plantae_Gmax
2    0.593808  0.578739         A0A0R0GAG6 6a6b TW-synth_TYW1 Plantae_Gmax
3   -0.425785  0.652042        A0A0R4J536 6a6b LipA-like_LipA Plantae_Gmax
4    0.817065 -0.163830          A0A0Z8EWX1 6a6b SPASM_SuiB Bacteria_Ssuis
..        ...       ...                                                ...
174  0.559930  0.627164        Q8RXN5 6a6b TW-synth_TYW1 Plantae_Athaliana
175  0.872359 -0.118711        Q8WXG1 6a6b Viperin_Viperin Metazoa_Hsapien
176 -0.235806 -0.786328        Q91WE6 6a6b MTTase_CDKAL1 Metazoa_Mmusculus
177  0.273615 -0.311784          Q93XX3 6a6b MTaseA_RlmN Plantae_Athaliana
178 -0.889401  0.024341        Q93ZR1 7a6b Elp-like_Elp3 Plantae_Athaliana

[179 rows x 3 columns]
3 annotations found: ['proteins_id', 'accession', 'label'].


### Create interactive plot

In [12]:
# Construction of custom color palette  

# Load existing colour palette (must be a .json file)
colour_json = 'examples/wayland_datasets/radical_sam/radicalsam_colors.json'
with open(colour_json, 'r') as file:
    colour_palette = json.load(file)

# Also the option of manually creating your own colour palette


In [13]:
# OPTIONS =================================================
#Here you can set different parameters to color & annotate the resulting projection:
title = "PM projection of radical sams with embeddings" 
#----------------------------------------------------------
# Select the coloring from annotation .csv file:
labels_name = "label"
# Select classes to label among the "labels_name" or "second_labels_name" column (comma separated list):
second_labels_name = ""
labels_text = []
show_text = False
#----------------------------------------------------------
# Use a custom color palette:
color_palette = colour_palette #Default: None
use_custom_palette = True
#==========================================================


#Check projection visualization parameters
#Labels name
if labels_name == "":
    labels_name = None
elif labels_name not in annotation_names:
    raise NameError(f"labels_name {labels_name} is not in the availables annotations.\nAvailables annotations: {annotation_names}")
#Second labels name
if second_labels_name == "":
    second_labels_name = None
elif second_labels_name not in annotation_names:
    raise NameError(f'"second_labels_name" {second_labels_name} is not in the availables annotations.\nAvailables annotations: {annotation_names}')

if not use_custom_palette:
    color_palette = None

#Plot graph
fig = plot_embedding_interactive(df_embedding, 
                                 labels_name = labels_name,
                                 second_labels_name = second_labels_name, 
                                 show_text = show_text,
                                 labels_text = labels_text,
                                 color_palette = color_palette, 
                                 title = title, 
                                 fontsize = 11)
fig.show()

In [ ]:
# OPTIONS =================================================
#Here you can set different parameters to color & annotate the resulting projection:
title = "PM projection of thioredoxins colored by species"
#----------------------------------------------------------
# Select the coloring from annotation .csv file:
labels_name = "Species"
# Select classes to label among the "labels_name" or "second_labels_name" column (comma separated list):
second_labels_name = ""
labels_text = []
show_text = False
#----------------------------------------------------------
# Use a custom color palette:
color_palette = trx_palette #Default: None
use_custom_palette = True
#==========================================================


#Check projection visualization parameters
#Labels name
if labels_name == "":
    labels_name = None
elif labels_name not in annotation_names:
    raise NameError(f"labels_name {labels_name} is not in the availables annotations.\nAvailables annotations: {annotation_names}")
#Second labels name
if second_labels_name == "":
    second_labels_name = None
elif second_labels_name not in annotation_names:
    raise NameError(f'"second_labels_name" {second_labels_name} is not in the availables annotations.\nAvailables annotations: {annotation_names}')

if not use_custom_palette:
    color_palette = None

#Plot graph
fig = plot_embedding_interactive(df_embedding, 
                                 labels_name = labels_name,
                                 second_labels_name = second_labels_name, 
                                 show_text = show_text,
                                 labels_text = labels_text,
                                 color_palette = color_palette, 
                                 title = title, 
                                 fontsize = 11)
fig.show()

### Save plot to file

In [14]:
# OPTIONS =================================================
output_name = "radical_sam_w_embeddings"
output_format = "html" #Format availables: ["png", "html", "pdf", "svg"]
#==========================================================


if output_format != "html":
    fig.write_image(f"{output_name}.{output_format}", engine="kaleido")
else:
    fig.write_html(f"{output_name}.{output_format}")